# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import isnan, when, count, col,countDistinct
from pyspark.sql.types import ArrayType, BooleanType, LongType, FloatType, IntegerType, DateType,StringType
from pyspark.sql import types as t


/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify project") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [4]:
#Load dataset
path = "mini_sparkify_event_data.json"
sparkify_df = spark.read.json(path)

In [5]:
#View the schema
sparkify_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
#View first 5 rows
sparkify_df.show(5)

+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|          artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|  Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Boston-Cambridge-...|   PUT|NextSong|1538331630000| 

In [7]:
#Number of rows Before doing any transformations
sparkify_df.count()


286500

In [8]:
#Check for duplicates
sparkify_df.groupBy(sparkify_df.columns)\
    .count()\
    .where(F.col('count') > 1)\
    .select(F.sum('count'))\
    .show()

+----------+
|sum(count)|
+----------+
|      null|
+----------+



In [9]:
#Check for missing values
sparkify_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in sparkify_df.columns]).show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId| song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
| 58392|   0|     8346|  8346|            0|    8346| 58392|    0|    8346|     0|   0|        8346|        0|58392|     0|  0|     8346|     0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+



In [10]:
#Cast userID to integer datatype
sparkify_df = sparkify_df.withColumn("userId", sparkify_df ["userId"].cast(IntegerType()))

In [11]:
#Drop records without userids or sessionids
sparkify_df = sparkify_df.dropna(how = "any", subset = ["userId", "sessionId"])

In [12]:
#Drop culumns not needed for now
sparkify_df = sparkify_df.drop(*["artist","song","firstName", "lastName"])  

In [13]:
sparkify_df.select('userId').dropDuplicates().sort('userId').show()

+------+
|userId|
+------+
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
|    20|
|    21|
+------+
only showing top 20 rows



In [14]:
#Number of rows after doing some transformations
sparkify_df.count()

278154

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [15]:
#For a seleced set of columns, lets explore the unique value counts
columns=["auth",
"gender",
"level",
"location",
"method",
"page",
"userAgent",
"ts"]

for c in columns:
    sparkify_df.groupBy(c).count().orderBy("count").show()
    

+---------+------+
|     auth| count|
+---------+------+
|Cancelled|    52|
|Logged In|278102|
+---------+------+

+------+------+
|gender| count|
+------+------+
|     M|123576|
|     F|154578|
+------+------+

+-----+------+
|level| count|
+-----+------+
| free| 55721|
| paid|222433|
+-----+------+

+--------------------+-----+
|            location|count|
+--------------------+-----+
|Scranton--Wilkes-...|    6|
|  Corpus Christi, TX|   11|
|          Albany, OR|   23|
|   Memphis, TN-MS-AR|   62|
|Deltona-Daytona B...|   73|
|Birmingham-Hoover...|   75|
|Cincinnati, OH-KY-IN|   88|
|       Duluth, MN-WI|  107|
|  Port St. Lucie, FL|  117|
|Portland-Vancouve...|  119|
|      Montgomery, AL|  125|
|Omaha-Council Blu...|  187|
|          Monroe, LA|  216|
|Santa Maria-Santa...|  218|
|Virginia Beach-No...|  246|
|Seattle-Tacoma-Be...|  246|
|         Bozeman, MT|  254|
|          Dayton, OH|  259|
|          Muncie, IN|  292|
|         Pontiac, IL|  315|
+--------------------+-----+
o

In [16]:
# sparkify_df = sparkify_df.withColumn("ts_str",col("ts").cast(StringType()))
# sparkify_df = sparkify_df.withColumn("reg_str",col("registration").cast(StringType()))
# sparkify_df=sparkify_df.withColumn("ts_time",F.to_timestamp(sparkify_df.ts_str))
# sparkify_df=sparkify_df.withColumn("registration_time",F.to_timestamp(sparkify_df.reg_str))
# sparkify_df.printSchema()


# representing registration and timstamp by creating new columns
# tss = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime("%m-%d-%Y %H:%M:%S"))
# sparkify_df = sparkify_df.withColumn('updated_ts', tss('ts'))
# downgrade_value  = udf(lambda x: 1 if x == "Submit Downgrade" else 0, IntegerType())
# sparkify_df = sparkify_df.withColumn("downgrade_value", downgrade_value("page"))

# Converting timestamps to datetime from epoch time, to get the hour of the day
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).hour)
# Adding a new column called hour to our dataframe
sparkify_df = sparkify_df.withColumn("hour", get_hour(sparkify_df.ts))

In [18]:
sparkify_df.printSchema()

root
 |-- auth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- hour: string (nullable = true)



In [19]:
sparkify_df.show(5)

Py4JJavaError: An error occurred while calling o347.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 2061, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/spark-2.4.3-bin-spark-2.4.3-bin-hadoop2.8/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/spark-2.4.3-bin-spark-2.4.3-bin-hadoop2.8/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/spark-2.4.3-bin-spark-2.4.3-bin-hadoop2.8/python/lib/pyspark.zip/pyspark/serializers.py", line 345, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/spark-2.4.3-bin-spark-2.4.3-bin-hadoop2.8/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterato

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.